In [ ]:
!pip install pydub
!pip install sentencepiece
!pip install replicate
!pip install googletrans
!pip install translate

In [ ]:
!pip install yt_dlp

In [ ]:
import yt_dlp

def download_video_and_get_title(url):
    ydl_opts = {
        'format': 'best',
        'outtmpl': 'video.%(ext)s',
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        video_info = ydl.extract_info(url, download=True)  # Download the video and extract its info
        title = video_info.get('title', None)  # Get video title

    return title

download_video_and_get_title("https://www.youtube.com/watch?v=JR-J_4d2_0A")

In [ ]:
!pip install moviepy

In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip

clip = VideoFileClip("video.mp4")
audio = clip.audio
audio.write_audiofile("audio.wav")
path = "audio.wav"

In [ ]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving New recording 11.wav to New recording 11 (1).wav


In [ ]:
num_speakers = 2

language = 'Urdu'

model_size = 'large'

model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'

In [ ]:

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np


NameError: ignored

In [ ]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

In [ ]:
model = whisper.load_model(model_size)

In [ ]:

result = model.transcribe(path)
segments = result["segments"]

In [ ]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [ ]:
audio = Audio()

In [ ]:
def segment_embedding(segment):
    start = segment["start"]
    # Whisper overshoots the end timestamp in the last segment
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(path, clip)

    # Convert waveform to single channel
    waveform = waveform.mean(dim=0, keepdim=True)

    return embedding_model(waveform.unsqueeze(0))

In [ ]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)


embeddings = np.nan_to_num(embeddings)

NameError: ignored

In [ ]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w",encoding="UTF-8")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [ ]:
print(open('transcript.txt','r',encoding="UTF-8").read())


SPEAKER 1 0:00:00
ستی صاحب کی حال چاہے لیں آپ کے؟ 
SPEAKER 2 0:00:02
آہ لانیال میں بلکل ٹھیک ہوں آپ کیسے ہیں؟ 
SPEAKER 1 0:00:04
اپنا لیپٹر سمبھالیں گہر رہا ہے 
SPEAKER 2 0:00:06
آہ جی بہت شکریہ 


In [ ]:
# from transformers import AutoProcessor, SeamlessM4TModel
# processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-medium")
# model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-medium")

# def translate(text_to_translate, src_lang, tgt_lang):
#   text_inputs = processor(text = text_to_translate, src_lang=src_lang, return_tensors="pt")
#   output_tokens = model.generate(**text_inputs, tgt_lang=tgt_lang, generate_speech=False)
#   translated_text = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
#   return translated_text

In [ ]:
from translate import Translator

def translate(text, source_language, target_language):
    translator = Translator(to_lang=target_language, from_lang=source_language)
    translation = translator.translate(text)
    return translation

In [ ]:
# Create dialogues directly from the segments
# dialogues = [f"{segment['speaker']}: {segment['text']}" for segment in segments]

# create english translated dialouges directly
dialogues = [f"{segment['speaker']}: {translate(segment['text'], 'ur', 'en')}" for segment in segments]

In [ ]:
pip install replicate

In [ ]:
from getpass import getpass
import os
import replicate

# Set the Replicate API token using environment variable
os.environ["REPLICATE_API_TOKEN"] = "r8_DTRf1lFxRUzLDtCmE5CmpLEZEwAMpBD2pCfsL"

# Function to generate prompt for the entire set of dialogues
def process_diarization(dialogues, pre_prompt):
    # Use a secure method to handle API tokens (e.g., environment variables)
    api_token = os.environ.get("REPLICATE_API_TOKEN")

    if not api_token:
        api_token = getpass("Enter your Replicate API token: ")

    # Combine all dialogues into a single text
    all_dialogues = "\n".join(dialogues)

    # Construct the prompt
    prompt = f"{pre_prompt}\n{all_dialogues}\n Assistant: "

    # Make sure the model and prompt are appropriate for Laama2 summarization
    model_id = "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3"

    # Run the model
    iterator_output = replicate.run(model_id, input={"prompt": prompt})

    combined_text = ""
    for text in iterator_output:
        combined_text += text  # combines the output

    return combined_text.strip()  # Strip to remove trailing spaces

pre_prompt = 'Below is the conversation between speakers. Please summarize the whole conversation:'

# Process and summarize the entire set of dialogues
summary = process_diarization(dialogues, pre_prompt)

# Print or use the summary as needed
print(summary)


Sure, here's a summary of the conversation:

Speaker 1 asked Speaker 2 how he was doing, and Speaker 2 replied that he was fine. Speaker 1 then mentioned that his laptop was falling apart, and Speaker 2 expressed gratitude for the comment. The assistant acknowledged the exchange and offered to help with any issues related to the laptop.


In [ ]:
translatedSummary = translate(summary, 'en', 'ur')
print(translatedSummary)

یقینی طور پر، گفتگو کا خلاصہ یہ ہے:

اسپیکر 1 نے اسپیکر 2 سے پوچھا کہ وہ کیسا ہے، اور اسپیکر 2 نے جواب دیا کہ وہ ٹھیک ہے ۔ اسپیکر 1 نے پھر ذکر کیا کہ اس کا لیپ ٹاپ ٹوٹ رہا ہے، اور اسپیکر 2 نے تبصرے کے لئے اظہار تشکر کیا ۔ اسسٹنٹ نے تبادلے کا اعتراف کیا اور لیپ ٹاپ سے متعلق کسی بھی مسئلے میں مدد کرنے کی پیش کش کی ۔
